In [ ]:
#| hide
# check if in colab and install package as needed
![ -e /content ] && ! pip show fastai-torchgeo && pip install git+https://github.com/butchland/fastai-torchgeo.git
![ -e /content ] && ! pip show nbdev && pip install nbdev

# EuroSAT Land Use/Land Cover (LULC) Classification Tutorial
> finetune a torchgeo pretrained resnet model on the torchgeo EuroSAT dataset for LULC

[![](https://raw.githubusercontent.com/butchland/fastai-torchgeo/master/assets/colab.svg)](https://colab.research.google.com/github/butchland/fastai-torchgeo/blob/master/nbs/tutorial/03-eurosat-classification.ipynb)

This is tutorial on finetuning a [pretrained torchgeo resnet model](https://torchgeo.readthedocs.io/en/stable/api/models.html#resnet) on the [torchgeo EuroSAT dataset](https://torchgeo.readthedocs.io/en/stable/api/datasets.html#eurosat)
using the [fastai](https://docs.fast.ai) framework.

> Note: this tutorial assumes _some_ familiarity with the fastai deep learning package and will focus on torchgeo integration.

## Installation 

Install the package

```bash
pip install git+https://github.com/butchland/fastai-torchgeo.git
```

## Import the packages and download the EuroSAT dataset 

In [ ]:
from torchgeo.datasets import EuroSAT
from torchgeo.datamodules import EuroSATDataModule 
from torchgeo.models import ResNet18_Weights, resnet18

import fastai.vision.all as fv

from fastai_torchgeo.data import GeoImageBlock
from fastai_torchgeo.resnet import make_resnet_model, resnet_split

## Create a fastai datablock and fastai dataloaders  

In [ ]:
batch_size=64
num_workers = fv.defaults.cpus
dset_name = 'EuroSAT'

In [ ]:
dblock = fv.DataBlock(blocks=(GeoImageBlock(), fv.CategoryBlock()),
                      get_items=fv.get_image_files,
                      splitter=fv.RandomSplitter(valid_pct=0.2, seed=42),
                      get_y=fv.parent_label,
                      item_tfms=fv.Resize(64),
                      batch_tfms=[fv.Normalize.from_stats(EuroSATDataModule.mean, EuroSATDataModule.std)],
                     )

In [ ]:
cfg = fv.fastai_cfg()
data_dir = cfg.path('data')
dset_path = data_dir/dset_name

In [ ]:
datamodule = EuroSATDataModule(root=dset_path,batch_size=batch_size, num_workers=num_workers, download=True)

In [ ]:
%%time
datamodule.prepare_data()

In [ ]:
dblock.summary(dset_path, show_batch=True)

In [ ]:
dls = dblock.dataloaders(dset_path, bs=batch_size)

## Download the torchgeo pretrained resnet model and prepare a fastai compatible model

In [ ]:
pretrained = resnet18(ResNet18_Weights.SENTINEL2_ALL_MOCO, num_classes=10) # load pretrained weights

In [ ]:
model = make_resnet_model(pretrained, n_out=10)

## Create a fastai learner

In [ ]:
learn = fv.Learner(
    dls, 
    model,
    loss_func=fv.CrossEntropyLossFlat(),
    metrics=[fv.error_rate],
    splitter=resnet_split,
)
# freeze uses parameter groups created by `resnet_split` 
# to lock parameters of pretrained model except for the model head

learn.freeze()

In [ ]:
# note: only head parameter group is trainable (except BatchNorm layers w/ch are always trainable)
learn.summary()

## Train the model

In [ ]:
# learn.lr_find()

In [ ]:
learn.fine_tune(1)

In [ ]:
from fastai.callback.tracker import SaveModelCallback

In [ ]:
learn.fit_one_cycle(10, lr_max=slice(3e-3, 6e-6), cbs=[SaveModelCallback()])

## Post training (error analysis and setting up for inference)

In [ ]:
learn.recorder.plot_loss()

In [ ]:
learn.validate()

In [ ]:
learn.save('resnet18-finetune-eurosat')

In [ ]:
interp = fv.ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(8)